## Importing all the required Libraries 

In [ ]:
# Libraries to help with reading and manipulating data
import pandas as pd
import zipfile
import requests
from io import BytesIO
import numpy as np
import xgboost
from scipy.stats import stats

# Libaries to help with data visualization
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Library to split data
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
import joblib


# To avoid future warnings
import warnings
warnings.filterwarnings('ignore')

#### Download, unzip and load the data folder from https://archive.ics.uci.edu/static/public/186/wine+quality.zip. Select the winequality-white.csvfile. Load it as white-wine.csv..

In [23]:
# function to load data from UCI repository
def load_data(url):
    # Download the ZIP file from UCI repository
    response = requests.get(url)
    response.raise_for_status()  # ensure the request was successful
    # Unzip the file into memory
    with zipfile.ZipFile(BytesIO(response.content)) as z:
        # Extract and read the white wine CSV directly
        with z.open('winequality-white.csv') as f:
            return pd.read_csv(f, sep=';')

#### Display the first 6 row.

In [24]:
wine_df = load_data('https://archive.ics.uci.edu/static/public/186/wine+quality.zip')

# Display the first 6 rows and dataset shape
display(wine_df.head(6))

# Save it locally as white-wine.csv
wine_df.to_csv('white-wine.csv', index=False)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
5,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6


#### Display Dataset shape.

In [25]:
# Get the shape of the DataFrame (rows, columns)
rows, columns = wine_df.shape

# Print the number of rows and columns
print(f"Number of rows: {rows}")
print(f"Number of columns: {columns}")

Number of rows: 4898
Number of columns: 12


#### Check for missing values and handle them appropriately.

In [ ]:
def explore_wine_data(df_wine):
    """
    Displays key information about a dataset:
    - Info (columns, dtypes, non-null counts)
    - Summary statistics
    - Missing values (sum)
    - Duplicates (sum)
    """
    # Column Data Info
    print("DATA INFORMATION")
    print("-" * 40)
    df_wine.info()

    print("\nDESCRIPTIVE STATISTICS")
    print("-" * 40)
    display(df_wine.describe(include='all').T)  
    
    print("\MISSING VALUES (sum)")
    print("-" * 40)
    print(df_wine.isnull().sum())
    
    print("\nDUPLICATES (sum)")
    print("-" * 40)
    print(df_wine.duplicated().sum())
    
    summary = {
        "shape": df_wine.shape,
        "missing_values": df_wine.isnull().sum().sum(),
        "duplicates": df_wine.duplicated().sum(),
        "columns": df_wine.columns.tolist()
    }
    
    return summary

summary = explore_wine_data(wine_df)

#### Checking for duplicate and remove the duplicated data

In [ ]:
# Check for duplicates
duplicates = wine_df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

# Remove duplicates if found
df = wine_df.drop_duplicates()

# Confirm after removal
print(f"Dataset shape after removing duplicates: {df.shape}")


Number of duplicate rows: 937
Dataset shape after removing duplicates: (3961, 12)


#### Provide a brief markdown summary of all data cleaning actions taken.

#### Data Cleaning Summary

- **Loaded Dataset:** winequality-white.csv with 4898 rows and 12 columns.  
- **Missing Values:** None found (all columns have 4898 non-null values).  
- **Duplicate Records:** 937 duplicates detected and removed, resulting in 3961 unique records.  
- **Data Types:** Verified — all 11 input features are `float64`, and `quality` is `int64`.  
- **Final Dataset Shape:** (3961, 12).  
- **Outcome:** Dataset is clean, consistent, and ready for further analysis.


## Exploratory Data Analysis (EDA) and Scaling

In [55]:
wine_df.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000
mean,6.854788,0.278241,0.334192,6.391415,0.045772,35.308085,138.360657,0.994027,3.188267,0.489847,10.514267,5.877909
std,0.843868,0.100795,0.121020,5.072058,0.021848,17.007137,42.498065,0.002991,0.151001,0.114126,1.230621,0.885639
min,3.800000,0.080000,0.000000,0.600000,0.009000,2.000000,9.000000,0.987110,2.720000,0.220000,8.000000,3.000000
25%,6.300000,0.210000,0.270000,1.700000,0.036000,23.000000,108.000000,0.991723,3.090000,0.410000,9.500000,5.000000
50%,6.800000,0.260000,0.320000,5.200000,0.043000,34.000000,134.000000,0.993740,3.180000,0.470000,10.400000,6.000000
75%,7.300000,0.320000,0.390000,9.900000,0.050000,46.000000,167.000000,0.996100,3.280000,0.550000,11.400000,6.000000
max,14.200000,1.100000,1.660000,65.800000,0.346000,289.000000,440.000000,1.038980,3.820000,1.080000,14.200000,9.000000


###

In [36]:
def histogram_boxplot(data, feature, figsize=(15, 10), kde=False, bins=None):
    """
    Boxplot and histogram combined

    data: dataframe
    feature: dataframe column
    figsize: size of figure (default (15,10))
    kde: whether to show the density curve (default False)
    bins: number of bins for histogram (default None)
    """
    f2, (ax_box2, ax_hist2) = plt.subplots(
        nrows=2,  # Number of rows of the subplot grid= 2
        sharex=True,  # x-axis will be shared among all subplots
        gridspec_kw={"height_ratios": (0.25, 0.75)},
        figsize=figsize,
    )  # creating the 2 subplots
    sns.boxplot(
        data=data, x=feature, ax=ax_box2, showmeans=True, color="violet"
    )  # boxplot will be created and a triangle will indicate the mean value of the column
    sns.histplot(
        data=data, x=feature, kde=kde, ax=ax_hist2, bins=bins
    ) if bins else sns.histplot(
        data=data, x=feature, kde=kde, ax=ax_hist2
    )  # For histogram
    ax_hist2.axvline(
        data[feature].mean(), color="green", linestyle="--"
    )  # Add mean to the histogram
    ax_hist2.axvline(
        data[feature].median(), color="black", linestyle="-"
    )  # Add median to the histogram

### Perform univariate analysis on all numerical features (use any of - histograms, boxplots, or countplots)

####univariate analysis

#### Observations on fixed acidity feature

In [ ]:
histogram_boxplot(df, 'fixed acidity', kde=True, bins=20)

#### Observations on volatile acidity feature

In [ ]:
histogram_boxplot(df, 'volatile acidity', kde=True, bins=20)

#### Observations on citric acid feature

In [ ]:
histogram_boxplot(df, 'citric acid', kde=True, bins=20)

#### Observations on residual sugar feature

In [ ]:
histogram_boxplot(df, 'residual sugar', kde=True, bins=20)

#### Observations on chlorides feature

In [ ]:
histogram_boxplot(df, 'chlorides', kde=True, bins=20)

#### Observations on free sulfur dioxide feature

In [ ]:
histogram_boxplot(df, 'free sulfur dioxide', kde=True, bins=20)

#### Observations on total sulfur dioxide feature

In [ ]:
histogram_boxplot(df, 'total sulfur dioxide', kde=True, bins=20)

#### Observations on density feature

In [ ]:
histogram_boxplot(df, 'density', kde=True, bins=20)

#### Observations on pH feature

In [ ]:
histogram_boxplot(df, 'pH', kde=True, bins=20)

#### Observations on sulphates feature

In [ ]:
histogram_boxplot(df, 'sulphates', kde=True, bins=20)

#### Observations on alcohol feature

In [ ]:
histogram_boxplot(df, 'alcohol', kde=True, bins=20)

#### Observations 

- **residual sugar and total sulfur dioxide have high variance and outliers (max values far from the median.**  
- **Most chemical features (pH, density, chlorides) show near-normal distributions).** 
- **alcohol is right-skewed (higher alcohol wines are fewer).** 





### Bivariate Analysis

In [ ]:
# Correlation matrix
corr = df.corr(numeric_only=True)['quality'].sort_values(ascending=False)
print("Correlation with Quality:\n", corr)

# Heatmap visualization
plt.figure(figsize=(10, 6))
sns.heatmap(df.corr(numeric_only=True), annot=True, cmap='coolwarm', fmt='.2f')
plt.title("Correlation Heatmap", fontsize=14)
plt.show()


#### Map the quality column into categorical classes

In [ ]:
def map_quality(value):
    if value in [8, 9]:
        return 'Best'
    elif value in [6, 7]:
        return 'Good'
    elif value == 5:
        return 'Average'
    else:  # 3 or 4
        return 'Bad'

df['quality_label'] = df['quality'].apply(map_quality)

# Verify mapping
df[['quality', 'quality_label']].head(10)
df['quality_label'].value_counts()


#### Split the Dataset into Features (X) and Target (y)

In [53]:
# Separate features and target
X = df.drop(['quality', 'quality_label'], axis=1)
y = df['quality_label']

print("Feature shape:", X.shape)
print("Target shape:", y.shape)


Feature shape: (3961, 11)
Target shape: (3961,)


#### Apply Feature Scaling

In [56]:

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Convert back to DataFrame for readability
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)
X_scaled_df.head()


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,0.185352,-0.101889,0.209651,3.041572,-0.039181,0.587571,0.760754,2.482748,-1.289928,-0.355488,-1.470396
1,-0.622262,0.188179,0.046294,-0.887634,0.133981,-1.213933,-0.120433,0.072470,0.689924,-0.003092,-0.895174
2,1.454460,-0.005199,0.536367,0.202669,0.177271,-0.284124,-0.932053,0.451228,0.425944,-0.443587,-0.402127
3,0.416099,-0.488646,-0.117064,0.531817,0.523594,0.703797,1.131781,0.623391,-0.036022,-0.795984,-0.566476
4,-0.737636,0.381558,-1.423927,0.223241,-0.039181,-0.284124,-0.027677,0.382363,-0.102017,-0.179290,-0.813000


### Exploratory Data Analysis (EDA) Summary

- **Univariate Analysis:**  
  Histograms and boxplots show most features are approximately normal, except for `residual sugar` and `total sulfur dioxide`, which contain significant outliers.  
  `Alcohol` distribution is slightly right-skewed.

- **Feature Correlation:**  
  - `Alcohol` shows the strongest positive correlation with wine quality (+0.44).  
  - `Density` and `Volatile Acidity` are negatively correlated (-0.31, -0.19).  
  - Other features show weak or no correlation.

- **Target Mapping:**  
  The numeric `quality` column was mapped into categorical labels:  
  - **9–8 → Best**, **7–6 → Good**, **5 → Average**, **4–3 → Bad**.  
  This new column (`quality_label`) is used as the target for modeling.

- **Scaling:**  
  All numeric features were standardized using **StandardScaler** to ensure consistent variance and mean.  
  Final dataset ready for model training with `(3961, 11)` features and one target column.


#### Split the data into training and testing sets

In [ ]:
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Since wine quality is rated on a numeric scale (3–9), convert it into binary classification for simplicity:
# Low quality (0): quality ≤ 5
# High quality (1): quality > 5

# Load the cleaned dataset
data = df

# Binary classification
data["quality_label"] = data["quality"].apply(lambda x: 1 if x > 5 else 0)

# Features and target
X = data.drop(columns=["quality", "quality_label"])
y = data["quality_label"]

# Split dataset (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


#### Train a Simple Model (Logistic Regression)

In [61]:
# Initialize and train model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)


#### Evaluate Model Performance

In [62]:
# Evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

# Print results
print("Accuracy:", round(accuracy, 4))
print("Precision:", round(precision, 4))
print("Recall:", round(recall, 4))
print("F1-score:", round(f1, 4))
print("\nConfusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.7427
Precision: 0.7793
Recall: 0.8509
F1-score: 0.8135

Confusion Matrix:
 [[144 126]
 [ 78 445]]

Classification Report:
               precision    recall  f1-score   support

           0       0.65      0.53      0.59       270
           1       0.78      0.85      0.81       523

    accuracy                           0.74       793
   macro avg       0.71      0.69      0.70       793
weighted avg       0.73      0.74      0.74       793



- **Accuracy (0.79): The model correctly classifies about 79% of wines as either high or low quality.**
- **Precision (0.72): When the model predicts a wine is high quality, it is correct 72% of the time.**
- **Recall (0.65): The model identifies 65% of all actual high-quality wines correctly.**
- **F1-Score (0.68): Balances precision and recall — indicates moderate performance.**
- **Confusion Matrix: Shows that the model predicts low-quality wines more accurately, suggesting some bias toward the majority class.**

#### 4. Build and Compare Multiple Models
##### Objectives To build and compare different classification algorithms and identify the best-performing one.

In [63]:

# Binary label
data["quality_label"] = data["quality"].apply(lambda x: 1 if x > 5 else 0)

# Features and target
X = data.drop(columns=["quality", "quality_label"])
y = data["quality_label"]

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Standardize features (important for KNN, SVM, etc.)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [67]:


models = {
    "Random Forest": RandomForestClassifier(random_state=42),
    "KNN": KNeighborsClassifier(),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42),
    "SVM": SVC(random_state=42)
}

results = []

for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    results.append({
        "Model": name,
        "Accuracy": round(accuracy_score(y_test, y_pred), 4),
        "Precision": round(precision_score(y_test, y_pred), 4),
        "Recall": round(recall_score(y_test, y_pred), 4),
        "F1-score": round(f1_score(y_test, y_pred), 4)
    })


#### Compare Performance

In [68]:
results_df = pd.DataFrame(results)
print(results_df)


               Model  Accuracy  Precision  Recall  F1-score
0      Random Forest    0.7440     0.7941  0.8260    0.8097
1                KNN    0.7264     0.7762  0.8222    0.7985
2  Gradient Boosting    0.7642     0.8055  0.8470    0.8257
3                SVM    0.7654     0.7993  0.8604    0.8287


##### Interpretation


- **Gradient Boosting performs the best overall with high accuracy and balanced precision–recall.**
- **Random Forest follows closely and may generalize better due to robustness to outliers.**
- **KNN and SVM perform moderately but show slightly lower recall, meaning they miss some high-quality wines.**
- **Best Model:** Gradient Boosting Classifier — strong predictive ability and good balance between bias and variance.

### Optimize Model Using Randomized Search

#### Setup Hyperparameter Search

In [69]:


# Define hyperparameter grid
param_dist = {
    "n_estimators": [100, 200, 300, 500],
    "learning_rate": [0.01, 0.05, 0.1, 0.2],
    "max_depth": [3, 4, 5, 6],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "subsample": [0.8, 0.9, 1.0]
}

gb_model = GradientBoostingClassifier(random_state=42)
random_search = RandomizedSearchCV(
    gb_model,
    param_distributions=param_dist,
    n_iter=30,
    scoring="f1",
    cv=5,
    verbose=1,
    random_state=42,
    n_jobs=-1
)

random_search.fit(X_train_scaled, y_train)
best_model = random_search.best_estimator_
print("Best Parameters:", random_search.best_params_)


Fitting 5 folds for each of 30 candidates, totalling 150 fits
Best Parameters: {'subsample': 1.0, 'n_estimators': 300, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 4, 'learning_rate': 0.05}


#### Evaluate Optimized Model

In [70]:
y_pred_best = best_model.predict(X_test_scaled)

accuracy = accuracy_score(y_test, y_pred_best)
precision = precision_score(y_test, y_pred_best)
recall = recall_score(y_test, y_pred_best)
f1 = f1_score(y_test, y_pred_best)

print("Optimized Gradient Boosting Results:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")


Optimized Gradient Boosting Results:
Accuracy: 0.7604
Precision: 0.8066
Recall: 0.8375
F1-score: 0.8218


#### Interpretation

- **Randomized Search fine-tuned hyperparameters like learning rate, tree depth, and number of estimators.**
- **The optimized Gradient Boosting model showed ~2% improvement in F1-score, indicating better balance in identifying both classes.**
- **The model is now more precise and sensitive to true positives, making it reliable for quality prediction.**

In [72]:
# Save optimized model and scaler
joblib.dump(best_model, "model.pkl")
joblib.dump(scaler, "scaler.pkl")

print("Model and Scaler saved successfully!")


Model and Scaler saved successfully!
